# Machine Learning - Weighted KNN

# Author: Asad Mahmood

<a id='toc'> </a>
## Table of Contents

1. [**Weighted KNN Self Implemenation**](#Start)<br/>
2. [**Distance Fuctions to used in KNN implementation**](#DistFunc)<br/>
3. [**Weighted 3 - NN implemenatation**](#3NN) <br/>
4. [**Prediction of Labels of test points**](#Pred)<br/>
5. [**Accuray for each distance type**](#Accuracy)<br/>
6. [**Conclusion**](#Conclusion)<br/>

<a id='Start'> </a>
## Weighted KNN Self Implemenation

In [1]:
# Importing Libraries
import numpy as np
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.pyplot as plt
import pandas as pd
import scipy

Importing libraries that are gonna be used to implement this algorithim

In [2]:
# Importing the dataset
dataset = pd.read_csv('knnData.csv')

#Training Set extraction
train = dataset.iloc[:, [0, 1]].values
train_Labels = dataset.iloc[:, 2].values 

#Test Set Extraction
test = dataset.iloc[:, [3, 4]].values
test_Labels = dataset.iloc[:, 5].values

In the above cell I have imported the dataset as a dataframe and have divided it into train and test sets. I have also separated their labels. They are in the form of mupy arrays.

[Return to Table of Contents](#toc) <br/>

<a id='DistFunc'> </a>
## Distance Fuctions to used in KNN implementation

#### Manhattan Distance

In [3]:
def ManhattanDistance(x,y):
    dist = abs(x[0] - y[0]) + abs(x[1] - y[1])
    return dist

Here I have created a function for calculating Manhattan Distance. It is also know as $L_1$ Norm too.

#### Euclidian Distance

In [4]:
import math  

def EuclideanDistance(x,y):
    dist = math.sqrt(sum([(a - b) ** 2 for a, b in zip(x, y)]))
    return dist

Here I have created a function for calculating Euclidian Distance. It is also know as $L_2$ Norm too.

#### L Infinity Norm

In [5]:
def LInfinityNorm(x,y):
    dist = max(abs(x[0] - y[0]),abs(x[1] - y[1]))
    return dist

Here I have created a function for L Infinity Norm.

[Return to Table of Contents](#toc) <br/>

<a id='3NN'> </a>
## Weighted 3 - NN implemenatation

In [6]:
'''
This is a weighted KNN implementation and it takes in the following paramenters
    1. train : training set 
    2. train_labels : training set labels
    3. test : test point
    4. distanceFunction : We can pass a numeric value between 1 to 3 for the distance function. This fuction can calculate the K
                          nearest neighbours using "Manhattan Distance" which can be called by passing 1 as a parameter, 
                          "Euclidian Distance" can be used by passing 2 as a parameter and "L Infinity Norm" by passing 3 as a
                          parameter.
    5. k : A numeric value can be passed here to select the number of nearest neighbours to use.                     
'''

def weightedKnn(train, train_labels, test, distanceFunction, k):
    
    # This will store the k nearest neighbours in it along with their training labels 
    neighbours=[]

    # This loops through the training set points and calculates the distance in this case manhattan distance 
    # between the test point and all the training set points. Then it appends it to neighbours list along 
    # with the training set labels.
    
    if distanceFunction == 1:
        for trainPt in range(40):
            manhattan_dist = ManhattanDistance(train[trainPt],test)
            neighbours.append((manhattan_dist,train_labels[trainPt]))
    
    elif distanceFunction == 2:
        for trainPt in range(40):
            euclidian_dist = EuclideanDistance(train[trainPt],test)
            neighbours.append((euclidian_dist,train_labels[trainPt]))
            
    elif distanceFunction == 3:
        for trainPt in range(40):
            LInfinity_dist = LInfinityNorm(train[trainPt],test)
            neighbours.append((LInfinity_dist,train_labels[trainPt]))
    
    else:
        print("Wrong Distance function number entered. Try again in the range of 1 to 3.")
        
    # Now the neighbours list is sorted and only the top k neighbours are left behind for voting.
    neighbours = sorted(neighbours)[:k]

    # These variables will be used to store the cummilative weighted votes for each class/label.
    VotesClassA = 0    # This variable will store votes if the label is "-1"
    VotesClassB = 0    # This variable will store votes if the label is "1"
    
    # We will loop through the neighbours and check if they match a certain class then we will divide their vote
    # i.e "1" by their distance calculated earlier using the distance formula. The votes will be added and the 
    # label with higher value will be assigned to the test point.
    for n in neighbours: 
        if n[1] == -1: 
            VotesClassA += (1 / (n[0])**2) 
              
        elif n[1] == 1:  
            VotesClassB += (1 / (n[0])**2) 
              
    
    if VotesClassA>VotesClassB:
        return -1
    else:
        return +1

This is an implementation of Weighted KNN using Manhattan, Eulidian and L Infinity distance to measusre the distance between the test point and the training points. This function takes input of all the available training points, the labels of the afore mentioned training pints, one test point and the number of neighbours we need to look at to decide the label of the test point.

[Return to Table of Contents](#toc) <br/>

<a id='Pred'> </a>
## Prediction of Labels of test points

In [7]:
predicted_Labels_ManhattanDistance = []
predicted_Labels_EuclidianDistance = []
predicted_Labels_LInfinityNorm = []

for i in range(40):
    predicted_Labels_ManhattanDistance.append(weightedKnn(train, train_Labels, test[i], 1, 3))
    predicted_Labels_EuclidianDistance.append(weightedKnn(train, train_Labels, test[i], 2, 3))
    predicted_Labels_LInfinityNorm.append(weightedKnn(train, train_Labels, test[i], 3, 3))

Over here we predict the labels of the test points and store them separetly based on the type of distance function.

### Confusion Matrix

In [8]:
# Making the Confusion Matrix - Manhattan Distance
from sklearn.metrics import confusion_matrix
cm_MD = confusion_matrix(test_Labels, predicted_Labels_ManhattanDistance)
cm_MD

array([[24,  2],
       [ 3, 11]], dtype=int64)

Here we have the confusion matrix with the following results:
 + True Positive: 23 instances
 + True Negative: 11 instances
 + False Positive: 3 instances
 + False Negative: 3 instances

In [9]:
# Making the Confusion Matrix - Euclidian Distance
cm_ED = confusion_matrix(test_Labels, predicted_Labels_EuclidianDistance)
cm_ED

array([[23,  3],
       [ 4, 10]], dtype=int64)

Here we have the confusion matrix with the following results:
 + True Positive: 23 instances
 + True Negative: 11 instances
 + False Positive: 3 instances
 + False Negative: 3 instances

In [10]:
# Making the Confusion Matrix - L Infinity Norm
cm_LN = confusion_matrix(test_Labels, predicted_Labels_LInfinityNorm)
cm_LN

array([[24,  2],
       [ 3, 11]], dtype=int64)

Here we have the confusion matrix with the following results:
 + True Positive: 24 instances
 + True Negative: 11 instances
 + False Positive: 2 instances
 + False Negative: 3 instances

[Return to Table of Contents](#toc) <br/>

<a id='Accuracy'> </a>
## Accuray for each distance type

#### Manhattan Distance

In [11]:
accuracyPct_MD = ((cm_MD[0,0] + cm_MD[1,1])/(cm_MD[0,0] + cm_MD[0,1] + cm_MD[1,0] + cm_MD[1,1]))*100
print("The accuracy achieved with Manhattan distance is ", accuracyPct_MD, "Pct.")

The accuracy achieved with Manhattan distance is  87.5 Pct.


#### Euclidean Distance

In [12]:
accuracyPct_ED = ((cm_ED[0,0] + cm_ED[1,1])/(cm_ED[0,0] + cm_ED[0,1] + cm_ED[1,0] + cm_ED[1,1]))*100
print("The accuracy achieved with Euclidian distance is ", accuracyPct_ED, "Pct.")

The accuracy achieved with Euclidian distance is  82.5 Pct.


#### L Infinity Norm

In [13]:
accuracyPct_LN = ((cm_LN[0,0] + cm_LN[1,1])/(cm_LN[0,0] + cm_LN[0,1] + cm_LN[1,0] + cm_LN[1,1]))*100
print("The accuracy achieved with Euclidian distance is ", accuracyPct_LN, "Pct.")

The accuracy achieved with Euclidian distance is  87.5 Pct.


[Return to Table of Contents](#toc) <br/>

<a id='Conclusion'> </a>
## Conclusion

As seen through the accuracy rates from each distance type it is apparant that there is no difference between using **Manhattan distance** and **L Infinity Norm** as both have the same accuracy rate of **87.5%** whereas we see a decreased accuracy rate when using **Euclidean Distance** which is **82.5%**

[Return to Table of Contents](#toc) <br/>